In [1]:
# En primer lugar vamos a cargar las librerias necesarias.

import pandas as pd
import ta  # Biblioteca para calcular indicadores técnicos
import time
import dtale

print ("Librerias cargadas")

Librerias cargadas


In [2]:
# Cargar y preprocesar los datos

# URL del archivo CSV
url = "https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv"

# Cargar el CSV en un DataFrame
df = pd.read_csv(url, skiprows=1)

# Ahora, convertimos el timestamp de UNIX a datetime y filtramos los ultimos 4 años (aproximado por 4*365)
# Asegúrate de que el DataFrame tiene las columnas necesarias
# El dataset contiene: Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
# Convertir 'Unix' a timestamp para usarlo como índice

df['timestamp'] = pd.to_datetime(df['Unix'], unit='ms')
df.set_index('timestamp', inplace=True)

# Y ordenarlo cronológicamente de mas antiguo a mas moderno
df.sort_index(inplace=True)

# Filtrar los últimos 4 años para el backtest
end_time = df.index.max()
start_time = end_time - pd.Timedelta(days=5*365+60)  # Aproximación de 4 años + 60 dias, para obtener bien los indicadores
df = df[(df.index >= start_time) & (df.index <= end_time)]

# Resample a 4h
df = df.resample('4h').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume BTC': 'sum',  
    'Volume USDT': 'sum',  
    'tradecount': 'sum',  
})

print ("Datos cargados correctamente")

Datos cargados correctamente


In [3]:
# Calcular indicadores técnicos
# RSI
df['rsi'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()

# MACD
macd = ta.trend.MACD(close=df['Close'])
df['macd'] = macd.macd()
df['signal'] = macd.macd_signal()

# Bollinger Bands
bollinger = ta.volatility.BollingerBands(close=df['Close'], window=20)
df['bollinger_upper'] = bollinger.bollinger_hband()
df['bollinger_lower'] = bollinger.bollinger_lband()

# EMA50
df['ema50'] = ta.trend.ema_indicator(close=df['Close'], window=50)


# Y procedemos a filtrar los datos anteriores a la ventana deseada
end_time = df.index.max()
start_time = end_time - pd.Timedelta(days=4*365)  # Aproximación de 4 años
df = df[(df.index >= start_time) & (df.index <= end_time)]

print ("Indicadores calculados")

Indicadores calculados


In [4]:
# Generar señales (usando la estrategia anterior)
def generate_signals(df):
    # Crear columnas de señales inicializadas a 0
    df['long_entry'] = 0
    df['long_exit'] = 0
    df['short_entry'] = 0
    df['short_exit'] = 0

    last_feedback_time = time.time()
    for i in range(1, len(df)):
        # Mostrar progreso cada 5 segundos
        if time.time() - last_feedback_time >= 5:
            print(f"Generando señales: {i}/{len(df)} registros procesados...")
            last_feedback_time = time.time()

        # Condiciones para entrada en largo:
        # - RSI < 35 (ligera sobreventa) O precio <= banda de Bollinger inferior
        # - MACD > signal (momentum alcista)
        # - El precio actual > EMA50 (tendencia global alcista)
        if ((df['rsi'].iloc[i] < 35 or df['Close'].iloc[i] <= df['bollinger_lower'].iloc[i]) and
            df['Close'].iloc[i] < df['ema50'].iloc[i]):
            df.iloc[i, df.columns.get_loc('long_entry')] = 1

        # Salida de largo:
        # - RSI > 65 (algo sobrecomprado) O MACD < signal (pérdida de momentum)
        if (df['rsi'].iloc[i] > 70 and df['macd'].iloc[i] < df['signal'].iloc[i]):
            df.iloc[i, df.columns.get_loc('long_exit')] = 1

        # Entrada en corto:
        # - RSI > 65 (ligera sobrecompra) O precio >= banda de Bollinger superior
        # - MACD < signal (momentum bajista)
        # - El precio actual < EMA50 (tendencia global bajista)
        if ((df['rsi'].iloc[i] > 65 or df['Close'].iloc[i] >= df['bollinger_upper'].iloc[i]) and
            df['Close'].iloc[i] > df['ema50'].iloc[i]):
            df.iloc[i, df.columns.get_loc('short_entry')] = 1

        # Salida de corto:
        # - RSI < 35 (vuelve a zona baja), MACD > signal (cambio a momentum alcista)
        # Aquí podrías usar un "o" para salir en cualquiera de las dos condiciones,
        # pero usar "y" puede asegurar que realmente hay un cambio de momentum.
        # Ajustar según preferencias.
        if (df['rsi'].iloc[i] < 30 and df['macd'].iloc[i] > df['signal'].iloc[i]):
            df.iloc[i, df.columns.get_loc('short_exit')] = 1

    return df

# Aplicar la generación de señales
df = generate_signals(df)

In [5]:
# Backtesting con métricas

def backtest_with_details(df):
    balance = 10000  # Capital inicial
    position = 0     # 0 = sin posición, 1 = largo, -1 = corto
    entry_price = 0
    trade_count = 0
    win_count = 0
    loss_count = 0
    profit_loss = []
    max_drawdown = 0
    peak_balance = balance

    # Detalles de todas las operaciones
    operations = []

    # Métricas separadas para operaciones long y short
    long_trade_count = 0
    long_win_count = 0
    long_loss_count = 0
    long_profit_loss = []

    short_trade_count = 0
    short_win_count = 0
    short_loss_count = 0
    short_profit_loss = []

    df['status']="WAIT"
    df['operation_open_price']=0.0
    df['operation_max_price']=0.0

    for i in range(len(df)):
        current_date = df.index[i]

        # StopLoss

        if position == 1 and df['Close'].iloc[i] < df['operation_max_price'].iloc[i-1] * 0.90:
            df.iloc[i, df.columns.get_loc('long_exit')] = 1
            print ("cierre long por STOP")

        if position == -1 and df['Close'].iloc[i] > df['operation_max_price'].iloc[i-1] * 1.10:
            df.iloc[i, df.columns.get_loc('short_exit')] = 1
            print ("cierre short por STOP")

        # Abrir posición larga
        if df['long_entry'].iloc[i] == 1 and position == 0:
            position = 1
            entry_price = df['Close'].iloc[i]
            operations.append({
                "entry_date": current_date,
                "entry_price": entry_price,
                "type": "long",
                "exit_date": None,
                "exit_price": None,
                "profit": None
            })
            trade_count += 1
            long_trade_count += 1
            df.iloc[i, df.columns.get_loc('status')] = "LONG"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = entry_price
            df.iloc[i, df.columns.get_loc('operation_max_price')] = entry_price

        # Cerrar posición larga
        elif df['long_exit'].iloc[i] == 1 and position == 1:
            exit_price = df['Close'].iloc[i]
            trade_profit = (exit_price - entry_price) * position
            balance += trade_profit
            profit_loss.append(trade_profit)
            operations[-1].update({
                "exit_date": current_date,
                "exit_price": exit_price,
                "profit": trade_profit
            })
            df.iloc[i, df.columns.get_loc('status')] = "CLOSE"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = df['operation_open_price'].iloc[i-1]
            df.iloc[i, df.columns.get_loc('operation_max_price')] = max(df['operation_max_price'].iloc[i-1], df['High'].iloc[i])

            # Actualizar métricas globales
            if trade_profit > 0:
                win_count += 1
                long_win_count += 1
            else:
                loss_count += 1
                long_loss_count += 1
            long_profit_loss.append(trade_profit)
            position = 0

        # Mantener posicion larga
        elif position == 1:
            df.iloc[i, df.columns.get_loc('status')] = "LONG"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = df['operation_open_price'].iloc[i-1]
            df.iloc[i, df.columns.get_loc('operation_max_price')] = max(df['operation_max_price'].iloc[i-1], df['High'].iloc[i])

        # Abrir posición corta
        if df['short_entry'].iloc[i] == 1 and position == 0:
            position = -1
            entry_price = df['Close'].iloc[i]
            operations.append({
                "entry_date": current_date,
                "entry_price": entry_price,
                "type": "short",
                "exit_date": None,
                "exit_price": None,
                "profit": None
            })
            trade_count += 1
            short_trade_count += 1
            df.iloc[i, df.columns.get_loc('status')] = "SHORT"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = entry_price
            df.iloc[i, df.columns.get_loc('operation_max_price')] = entry_price          

        # Cerrar posición corta
        elif df['short_exit'].iloc[i] == 1 and position == -1:
            exit_price = df['Close'].iloc[i]
            trade_profit = (entry_price - exit_price) * abs(position)
            balance += trade_profit
            profit_loss.append(trade_profit)
            operations[-1].update({
                "exit_date": current_date,
                "exit_price": exit_price,
                "profit": trade_profit
            })
            df.iloc[i, df.columns.get_loc('status')] = "CLOSE"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = df['operation_open_price'].iloc[i-1]
            df.iloc[i, df.columns.get_loc('operation_max_price')] = min(df['operation_max_price'].iloc[i-1], df['Low'].iloc[i])

            # Actualizar métricas globales
            if trade_profit > 0:
                win_count += 1
                short_win_count += 1
            else:
                loss_count += 1
                short_loss_count += 1
            short_profit_loss.append(trade_profit)
            position = 0

        # Mantener posicion larga
        elif position == -1:
            df.iloc[i, df.columns.get_loc('status')] = "SHORT"
            df.iloc[i, df.columns.get_loc('operation_open_price')] = df['operation_open_price'].iloc[i-1]
            df.iloc[i, df.columns.get_loc('operation_max_price')] = min(df['operation_max_price'].iloc[i-1], df['Low'].iloc[i])

        # Calcular drawdown
        peak_balance = max(peak_balance, balance)
        drawdown = peak_balance - balance
        max_drawdown = max(max_drawdown, drawdown)

    # Métricas globales
    win_rate = win_count / trade_count if trade_count > 0 else 0
    avg_profit_loss = sum(profit_loss) / len(profit_loss) if profit_loss else 0

    # Métricas para operaciones long
    long_win_rate = long_win_count / long_trade_count if long_trade_count > 0 else 0
    long_avg_profit_loss = sum(long_profit_loss) / len(long_profit_loss) if long_profit_loss else 0

    # Métricas para operaciones short
    short_win_rate = short_win_count / short_trade_count if short_trade_count > 0 else 0
    short_avg_profit_loss = sum(short_profit_loss) / len(short_profit_loss) if short_profit_loss else 0

    return {
        "final_balance": balance,
        "trade_count": trade_count,
        "win_rate": win_rate,
        "average_profit_loss": avg_profit_loss,
        "max_drawdown": max_drawdown,
        "operations": operations,
        "long_trade_count": long_trade_count,
        "long_win_rate": long_win_rate,
        "long_average_profit_loss": long_avg_profit_loss,
        "short_trade_count": short_trade_count,
        "short_win_rate": short_win_rate,
        "short_average_profit_loss": short_avg_profit_loss,
        "df": df
    }

# Ejecutar el backtest
results = backtest_with_details(df)

cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre long por STOP
cierre long por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre long por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP
cierre short por STOP


In [6]:
# Mostrar resultados
print("Resultados del backtesting:")
print(f"Balance final: {results['final_balance']:.2f}")
print(f"Número de operaciones: {results['trade_count']}")
print(f"Tasa de éxito: {results['win_rate']:.2%}")
print(f"Ganancia/pérdida promedio: {results['average_profit_loss']:.2f}")
print(f"Máximo drawdown: {results['max_drawdown']:.2f}")
print("-- Operaciones LONG")
print(f"Total operaciones: {results['long_trade_count']:.2f}")
print(f"Tasa de éxito: {results['long_win_rate']:.2%}")
print(f"Ganancia/pérdida promedio: {results['long_average_profit_loss']:.2f}")
print("-- Operaciones SHORT")
print(f"Total operaciones: {results['short_trade_count']:.2f}")
print(f"Tasa de éxito: {results['short_win_rate']:.2%}")
print(f"Ganancia/pérdida promedio: {results['short_average_profit_loss']:.2f}")

# Mostrar detalles de las operaciones
print("\nDetalles de las operaciones:")
for op in results['operations']:

    exit_date = op['exit_date'] if op['exit_date'] is not None else ''
    exit_price = f"{op['exit_price']:.2f}" if op['exit_price'] is not None else ''
    profit = f"{op['profit']:.2f}" if op['profit'] is not None else ''

    print(f"Tipo: {op['type']} | Fecha entrada: {op['entry_date']} | Precio entrada: {op['entry_price']:.2f} | "
          f"Fecha salida: {exit_date} | "
          f"Precio salida: {exit_price} | "
          f"Ganancia: {profit}")

Resultados del backtesting:
Balance final: -134154.20
Número de operaciones: 139
Tasa de éxito: 32.37%
Ganancia/pérdida promedio: -1044.60
Máximo drawdown: 144950.31
-- Operaciones LONG
Total operaciones: 42.00
Tasa de éxito: 30.95%
Ganancia/pérdida promedio: -946.98
-- Operaciones SHORT
Total operaciones: 97.00
Tasa de éxito: 32.99%
Ganancia/pérdida promedio: -1085.86

Detalles de las operaciones:
Tipo: short | Fecha entrada: 2020-12-15 00:00:00 | Precio entrada: 19479.70 | Fecha salida: 2020-12-16 20:00:00 | Precio salida: 21335.52 | Ganancia: -1855.82
Tipo: short | Fecha entrada: 2020-12-17 00:00:00 | Precio entrada: 21753.26 | Fecha salida: 2020-12-20 16:00:00 | Precio salida: 23930.00 | Ganancia: -2176.74
Tipo: short | Fecha entrada: 2020-12-21 00:00:00 | Precio entrada: 23945.29 | Fecha salida: 2020-12-25 08:00:00 | Precio salida: 24574.40 | Ganancia: -629.11
Tipo: short | Fecha entrada: 2020-12-25 16:00:00 | Precio entrada: 24449.66 | Fecha salida: 2020-12-27 04:00:00 | Precio s

In [7]:
d = dtale.show(results['df'])

print (d._url)
print ("http://localhost:40000")

http://codespaces-301aa9:40000
http://localhost:40000
